# Quick fudge to provide sample data for regridded comparison

In [1]:
# Adjust environment to fix the iris-test-data location
import os
os.environ['OVERRIDE_TEST_DATA_REPOSITORY'] = "/data/users/itpp/git/iris-test-data/test_data"

In [2]:
import numpy as np

import iris
from iris import load_cube
import iris.coord_systems as icrs
from iris.experimental.ugrid import PARSE_UGRID_ON_LOAD
from iris.tests import get_data_path

In [3]:
import iris.config

In [4]:
print(iris.config.TEST_DATA_DIR)

/data/users/itpp/git/iris-test-data/test_data


In [5]:
# Get a sample datafile with a C48 cubesphere mesh.
file_path = get_data_path(["NetCDF", "unstructured_grid", "lfric_surface_mean.nc"])

with PARSE_UGRID_ON_LOAD.context():
  cube_rotatedcs = load_cube(file_path, 'rainfall_flux')
  # Simply make a separate copy with its own separate mesh (for now)
  orig_cube_copy = load_cube(file_path, 'rainfall_flux')

assert cube_rotatedcs.mesh == orig_cube_copy.mesh
assert cube_rotatedcs.mesh is not orig_cube_copy.mesh
assert cube_rotatedcs.mesh.node_coords.node_x is not orig_cube_copy.mesh.node_coords.node_x

print(cube_rotatedcs)
print(f'\nCubesphere "N"={int(np.sqrt(cube_rotatedcs.shape[-1] / 6))}')

rainfall_flux / (kg m-2 s-1)        (-- : 1; -- : 13824)
    Mesh coordinates:
        latitude                        -       x
        longitude                       -       x
    Auxiliary coordinates:
        time                            x       -
    Cell methods:
        mean                        time (300 s)
        mean                        time_counter
    Attributes:
        Conventions                 'UGRID'
        description                 'Created by xios'
        interval_operation          '300 s'
        interval_write              '1 d'
        name                        'lfric_surface'
        online_operation            'average'
        timeStamp                   '2020-Feb-07 16:23:14 GMT'
        title                       'Created by xios'
        uuid                        '489bcef5-3d1c-4529-be42-4ab5f8c8497b'

Cubesphere "N"=48


In [6]:
# Rotate the mesh coordinates to produce a more "interesting" mesh for our regrid target.
crs_plain = icrs.GeogCS(6.e6)
crs_rot = icrs.RotatedGeogCS(35.7, 0.45, ellipsoid=crs_plain)
ccrs_plain = crs_plain.as_cartopy_crs()
ccrs_rot = crs_rot.as_cartopy_crs()


# we need to grab the node locations and transform them.
cube_nodes_xco = cube_rotatedcs.mesh.node_coords.node_x
cube_nodes_yco = cube_rotatedcs.mesh.node_coords.node_y

lat_pts = cube_nodes_yco.points
lat_bds = cube_nodes_yco.bounds
lon_pts = cube_nodes_xco.points
lon_bds = cube_nodes_xco.bounds

print(f'Original lon-pts minmax = {lon_pts.min(), lon_pts.max()}')
print(f'Original lat-pts minmax = {lat_pts.min(), lat_pts.max()}')

xxx_pts = ccrs_rot.transform_points(ccrs_plain, lon_pts, lat_pts)
# xxx_bds = ccrs_rot.transform_points(ccrs_plain, lon_bds, lat_bds)
print(f'transformed, shape = {xxx_pts.shape}')


lon_pts_tx = xxx_pts[:, 0]
lat_pts_tx = xxx_pts[:, 1]
print(f'Tranformed lon-pts minmax = {lon_pts_tx.min(), lon_pts_tx.max()}')
print(f'Tranformed lat-pts minmax = {lat_pts_tx.min(), lat_pts_tx.max()}')

# Force-reset the coords
cube_nodes_xco.points = lon_pts_tx
cube_nodes_yco.points = lat_pts_tx
assert cube_rotatedcs.mesh != orig_cube_copy.mesh

Original lon-pts minmax = (0.0, 358.125)
Original lat-pts minmax = (-90.0, 90.0)
transformed, shape = (13826, 3)
Tranformed lon-pts minmax = (-180.0, 179.98227674550594)
Tranformed lat-pts minmax = (-89.62677763516267, 89.62677763516267)


In [7]:
# Display these both ??

from geovista import Transform
from geovista.geoplotter import GeoPlotter
from geovista import theme

In [8]:
def pv_from_unstructcube(cube):
    return Transform.from_unstructured(
        xs=cube.mesh.node_coords.node_x.points,
        ys=cube.mesh.node_coords.node_y.points,
        connectivity=cube.mesh.face_node_connectivity.indices_by_location(),
        data=cube.data,
    )    

In [9]:
my_polydata = pv_from_unstructcube(cube_rotatedcs)

my_plotter = GeoPlotter()
my_plotter.theme.font.color = (0.5,) * 3
my_plotter.theme.edge_color = 'pink'
my_plotter.add_coastlines()
my_plotter.add_mesh(my_polydata, show_edges=True)
my_plotter.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [10]:
# Load a basic (small) UM file for its lat-lon grid.
fp2 = get_data_path(['FF', 'n48_multi_field'])
import iris.exceptions
iris.exceptions.IgnoreCubeException()

# (A bit awkward, as there are two cubes with the same phenom name : one is a mean)
def no_cellmeth(cube, field, filename):
    if cube.cell_methods:
        raise iris.exceptions.IgnoreCubeException()

latlon_cube = iris.load_cube(fp2, 'air_temperature', callback=no_cellmeth)
orog_cube = iris.load_cube(fp2, 'surface_altitude')

In [11]:
print(latlon_cube)

air_temperature / (K)               (latitude: 73; longitude: 96)
    Dimension coordinates:
        latitude                             x              -
        longitude                            -              x
    Scalar coordinates:
        forecast_period             0.0 hours
        forecast_reference_time     2011-07-11 00:00:00
        height                      1.5 m
        time                        2011-07-11 00:00:00
    Attributes:
        STASH                       m01s03i236
        source                      'Data from Met Office Unified Model'
        um_version                  '8.2'


In [12]:
# Routine to create a Polydata from a structured 2d cube.
def pv_from_structcube(cube):
    xco = cube.coord(axis='x')
    yco = cube.coord(axis='y')
    for co in (xco, yco):
        if not co.has_bounds():
            co.guess_bounds()
    return Transform.from_1d(xs=xco.bounds, ys=yco.bounds, data=cube.data)

In [13]:
# Routine to display a single mesh (Polydata) with PyVista
def display_one(mesh):
    my_plotter = GeoPlotter()
    my_plotter.theme.font.color = (0.5,) * 3
    my_plotter.theme.edge_color = 'pink'
    my_plotter.add_coastlines()
    my_plotter.add_mesh(mesh, show_edges=True)
    my_plotter.show()

In [14]:
# Show the latlon sample data (grid)
latlon_mesh = pv_from_structcube(latlon_cube)
display_one(latlon_mesh)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [15]:
# Use iris-esmf-regrid for regridding
import esmf_regrid as ief

In [16]:
# Fetch some nice-looking original LFRic (unstructured) test data.
file_path = get_data_path(["NetCDF", "unstructured_grid", "lfric_ngvat_3D_1t_full_level_face_grid_main_area_fraction_unit1.nc"])

from iris.experimental.ugrid.load import PARSE_UGRID_ON_LOAD
with PARSE_UGRID_ON_LOAD.context():
    testdata_cube = load_cube(file_path, "area_fraction")

print(testdata_cube)

area_fraction / (1)                 (-- : 1; full_levels: 39; -- : 864)
    Dimension coordinates:
        full_levels                     -               x        -
    Mesh coordinates:
        latitude                        -               -        x
        longitude                       -               -        x
    Auxiliary coordinates:
        time                            x               -        -
    Cell methods:
        point                       time (300 s)
    Attributes:
        Conventions                 'UGRID'
        description                 'Created by xios'
        interval_operation          '300 s'
        interval_write              '21600 s'
        name                        'lfric_ngvat_3D_1t_full_level_face_grid_main_area_fraction_unit1'
        online_operation            'instant'
        timeStamp                   '2020-Oct-18 21:20:19 GMT'
        title                       'Created by xios'
        uuid                        'e9218bc2-96

In [17]:
# Display a single nice-looking layer
onelayer_mesh = pv_from_unstructcube(testdata_cube[0,10])

display_one(onelayer_mesh)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [18]:
# Regrid the test data onto the latlon grid.
import esmf_regrid as ief
from esmf_regrid.experimental.unstructured_scheme import MeshToGridESMFRegridder
from esmf_regrid.experimental.unstructured_scheme import GridToMeshESMFRegridder

In [19]:
regridder = MeshToGridESMFRegridder(testdata_cube, latlon_cube)
testdata_on_latlon_cube = regridder(testdata_cube)

In [20]:
# Show that
testdata_on_latlon_mesh = pv_from_structcube(testdata_on_latlon_cube[0,10])
display_one(testdata_on_latlon_mesh)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [21]:
# Now regrid this "back" onto the basic cubesphere mesh + compare those
test_backon_c12_cube = GridToMeshESMFRegridder(testdata_on_latlon_cube, testdata_cube)(testdata_on_latlon_cube)

In [22]:
test_backon_c12_mesh = pv_from_unstructcube(test_backon_c12_cube[0,10])

In [23]:
test_on_rotatedC48_cube = GridToMeshESMFRegridder(testdata_on_latlon_cube, cube_rotatedcs)(testdata_on_latlon_cube)

In [24]:
test_on_rotatedC48_mesh = pv_from_unstructcube(test_on_rotatedC48_cube[0,10])

In [25]:
my_plotter = GeoPlotter(shape=(2, 2))

my_plotter.subplot(0, 0)
my_plotter.add_text("Original mesh data", font_size=12)
my_plotter.add_coastlines()
my_plotter.add_mesh(onelayer_mesh, show_edges=True)

my_plotter.subplot(0, 1)
my_plotter.add_text("Regridded onto latlon", font_size=12)
my_plotter.add_coastlines()
my_plotter.add_mesh(testdata_on_latlon_mesh, show_edges=True)

my_plotter.subplot(1, 0)
my_plotter.add_text("Latlon -> back to C12", font_size=12)
my_plotter.add_coastlines()
my_plotter.add_mesh(test_backon_c12_mesh, show_edges=True)

my_plotter.subplot(1, 1)
my_plotter.add_text("Latlon -> rotated C48", font_size=12)
my_plotter.add_coastlines()
my_plotter.add_mesh(test_on_rotatedC48_mesh, show_edges=True)

my_plotter.link_views()
my_plotter.camera.position = [0, -2.5, 2.5]
my_plotter.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [26]:
# Save all the files shown in the above.

iris.save(testdata_cube, 'data_orig.nc')  # the 'area_fraction' cube from the original file
iris.save(testdata_on_latlon_cube, 'data_latlon.nc')
iris.save(test_backon_c12_cube, 'latlon_regridded_C12.nc')
iris.save(test_on_rotatedC48_cube, 'latlon_regridded_rotC48.nc')